# 1. Generate ROI tiles

In [71]:
from os.path import join, basename
from os import makedirs,cpu_count
from glob import glob
from concurrent.futures import ProcessPoolExecutor
import time

In [ ]:
wdir = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/"
lidar_dpath = join(wdir, 'LiDAR')
tiles_dpath = join(wdir, 'TILES')

ds_dpath = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TILES12B/N13E103/"
vrt_files = ds_files = glob(f'{ds_dpath}/*.tif'); print(len(ds_files))

cpus = int(cpu_count() - 3)
basefiles = lidar_files = glob(f'{lidar_dpath}/*.tif')
lidar_files

In [53]:
import os
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
import numpy as np
import geopandas as gpd



def gdal_polygonize(tif_path, gpkg_path):
    cmd = f'gdal_polygonize.py {tif_path} -b 1 -f "GPKG" {gpkg_path}'# OUTPUT DN'
    os.system(cmd)

def dissolve_features(input_gpkg,output_gpkg):
    #input_layer, 
    """
    Dissolve all features in a GeoPackage layer into a single geometry.

    :param input_gpkg: Path to the input GeoPackage file.
    :param input_layer: Name of the layer to dissolve.
    :param output_gpkg: Path to the output GeoPackage file with dissolved features.
    """
    # Load the layer from the GeoPackage
    gdf = gpd.read_file(input_gpkg)#, layer=input_layer)
    
    # Dissolve all features into a single geometry
    dissolved = gdf.dissolve()
    
    # Save the dissolved geometry to a new GeoPackage
    dissolved.to_file(output_gpkg, driver='GPKG') #layer='dissolved',
    print(f"Dissolved features saved to {output_gpkg}")

def buffer_features(input_gpkg,output_gpkg, buffer_distance):
    """
    Apply a buffer to the dissolved features in a GeoPackage layer.

    :param input_gpkg: Path to the input GeoPackage file with dissolved features.
    :param input_layer: Name of the layer to buffer.
    :param output_gpkg: Path to the output GeoPackage file with buffered features.
    :param buffer_distance: Distance for the buffer operation.
    """
    # Load the dissolved layer from the GeoPackage
    gdf = gpd.read_file(input_gpkg)#, layer=input_layer)
    
    # Apply the buffer operation
    buffered = gdf.buffer(buffer_distance)
    
    # Create a new GeoDataFrame to store the buffered geometry
    buffered_gdf = gpd.GeoDataFrame(geometry=buffered, crs=gdf.crs)
    
    # Save the buffered geometry to a new GeoPackage
    buffered_gdf.to_file(output_gpkg,driver='GPKG')# layer='buffered', 
    print(f"Buffered features saved to {output_gpkg}")

def save_to_gpkg(gdf, output_path):
    gdf.to_file(output_path, driver='GPKG')

def raster2vectorcutline(raster_path, gpkg_path):
    # Convert raster to polygons

    gpkg_path_int = gpkg_path.replace('.gpkg', '_P.gpkg')
    fb = gpkg_path_int.replace('.gpkg','_B.gpkg')
    gdal_polygonize(raster_path, gpkg_path_int)
    buffer_features(gpkg_path_int, fb, 0)
    dissolve_features(fb,gpkg_path)
    os.remove(gpkg_path_int)
    os.remove(fb)
 
    print("Process completed. Output saved to:", gpkg_path)


for j,fi in enumerate(basefiles):
    fo = fi.replace('.tif', '.gpkg')
    print(fi,'\n',fo)
    #raster2vectorcutline(fi,fo)

In [ ]:
import rasterio
from rasterio.mask import mask
import fiona
import os

def clip_cultilinex(vpath,rpath, fopath):
    cmd = f'gdalwarp -cutline {vpath} -crop_to_cutline -dstalpha {rpath} {fopath}'
    os.system(cmd)

def clip_cultiline(vpath, rpath, fopath):
  # Open the vector file
  with fiona.open(vpath, "r") as shapefile:
      # Extract the geometry from the vector file
      shapes = [feature["geometry"] for feature in shapefile]

  # Open the raster file
  with rasterio.open(rpath) as src:
      # Clip the raster with the geometry
      out_image, out_transform = mask(src, shapes, crop=True)
      out_meta = src.meta.copy()

      # Update the metadata to reflect the new dimensions
      out_meta.update({
          "driver": "GTiff",
          "height": out_image.shape[1],
          "width": out_image.shape[2],
          "transform": out_transform
      })

      # Write the clipped raster to a new file
      with rasterio.open(fopath, "w", **out_meta) as dest:
          dest.write(out_image)



def get_out_params(tiles_dpath,bfile,src_dataset):
    dname = basename(bfile).replace('.gpkg','').split('_')[-1]
    dst_dpath = join(tiles_dpath,dname)
    makedirs(dst_dpath, exist_ok=True)
    dst_dataset = join(dst_dpath,basename(src_dataset))
    return dst_dataset


gpkg_files = glob(f'{lidar_dpath}/*.gpkg')
gpkg_files,lidar_dpath

if __name__ == '__main__':
    ti = time.perf_counter()
    with ProcessPoolExecutor(cpus) as PPE:
        for i,vrt in enumerate(vrt_files):
            src_dataset = vrt
            #print(vrt)
            for j,bfile in enumerate(gpkg_files):
                dst_dataset = get_out_params(tiles_dpath,bfile,src_dataset)
                #clip_cultiline(src_dataset,bfile,dst_dataset)

               
                # PPE.submit(
                #    clip_cultilinex,bfile,src_dataset,dst_dataset
                # )
 
               
    tf = time.perf_counter() - ti
    print(f'run.time : {tf/60}  min(s)')


# 2. Generate patches

In [74]:
import os
import itertools
import rasterio as rio
from rasterio import windows
from multiprocessing import Pool, cpu_count
from glob import glob

def get_tiles(ds, tile_width, tile_height, overlap):
  nols, nrows = ds.meta['width'], ds.meta['height']
  xstep = tile_width - overlap
  ystep = tile_height - overlap
  for x in range(0, nols, xstep):
      if x + tile_width > nols:
          x = nols - tile_width
      for y in range(0, nrows, ystep):
          if y + tile_height > nrows:
              y = nrows - tile_height
          window = windows.Window(x, y, tile_width, tile_height)
          transform = windows.transform(window, ds.transform)
          yield window, transform

def process_tile(args):
  window, transform, src_meta, in_path, out_path, output_filename = args
  metadata = src_meta.copy()
  metadata['transform'] = transform
  metadata['width'], metadata['height'] = window.width, window.height
  out_filepath = os.path.join(out_path, output_filename.format(window.col_off, window.row_off))
  
  with rio.open(in_path) as src:
      with rio.open(out_filepath, 'w', **metadata) as dst:
          dst.write(src.read(window=window))

def calculate_offset(width, height, tile_width, tile_height, stride_x, stride_y):
  X = [x for x in range(0, width, stride_x)]
  Y = [y for y in range(0, height, stride_y)]
  offsets = list(itertools.product(X, Y))
  return offsets

In [75]:
psize = 64
WDIR = "/media/ljp238/12TBWolf/ARCHIEVE/OUT/TONLESAP/TILES/"
tile_width = psize
tile_height = psize
overlap = 0
output_filename = 'tile_{}_{}.tif'
tilenames = os.listdir(WDIR)
tilenames

['TSAP1', 'TSAP2']

In [ ]:
Stile = "N13E103"
for tilename in tilenames:
    tif_dpath = os.path.join(WDIR, tilename)
    patch_dpath = os.path.join(tif_dpath, f'P{psize}')
    os.makedirs(patch_dpath, exist_ok=True)
    tif_files = glob(f'{tif_dpath}/*.tif')
    for in_path in tif_files:
        out_path = os.path.join(patch_dpath, os.path.splitext(os.path.basename(in_path))[0])
        out_path = out_path.replace(f'{Stile}_','')
        print(out_path)
        os.makedirs(out_path, exist_ok=True)

        with rio.open(in_path) as src:
            src_meta = src.meta.copy()
            width, height = src_meta['width'], src_meta['height']
            stride_x = tile_width - overlap
            stride_y = tile_height - overlap
            offsets = calculate_offset(width, height, tile_width, tile_height, stride_x, stride_y)
            tiles = [(windows.Window(x, y, tile_width, tile_height), windows.transform(windows.Window(x, y, tile_width, tile_height), src.transform)) for x, y in offsets]

        args = [(window, transform, src_meta, in_path, out_path, output_filename) for window, transform in tiles]

        with Pool(processes=cpu_count()) as pool:
            pool.map(process_tile, args)

# 3. Generate df paths with nulls